In [1]:
import os
import logging
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from scipy import stats
from tqdm import tqdm
from datetime import datetime, timedelta
import warnings
from sklearn.linear_model import LinearRegression
import requests
from io import StringIO

def plot_performance(strategy_returns, benchmark_returns=None, title='Strategy Performance'):
    """
    Genera gráfico de rendimiento.
    
    Args:
        strategy_returns (Series): Serie con retornos de la estrategia
        benchmark_returns (Series, optional): Serie con retornos del benchmark
        title (str): Título del gráfico
        
    Returns:
        None
    """
    try:
        plt.figure(figsize=(12, 6))
        
        # Calcular retorno acumulado
        strategy_cumulative = (1 + strategy_returns).cumprod() - 1
        
        # Graficar retorno acumulado de la estrategia
        plt.plot(strategy_cumulative.index, strategy_cumulative.values, label='Strategy')
        
        # Si hay benchmark, graficar también
        if benchmark_returns is not None:
            # Alinear fechas
            aligned_benchmark = benchmark_returns.reindex(strategy_returns.index)
            
            # Calcular retorno acumulado del benchmark
            benchmark_cumulative = (1 + aligned_benchmark).cumprod() - 1
            
            # Graficar retorno acumulado del benchmark
            plt.plot(benchmark_cumulative.index, benchmark_cumulative.values, label='Benchmark')
        
        # Añadir título y etiquetas
        plt.title(title)
        plt.xlabel('Date')
        plt.ylabel('Cumulative Return')
        plt.legend()
        plt.grid(True)
        
        # Guardar gráfico
        plt.savefig(f'./artifacts/results/figures/{title.replace(" ", "_")}.png')
        plt.close()
    
    except Exception as e:
        logging.error(f"Error generando gráfico de rendimiento: {str(e)}")

def plot_drawdown(returns, title='Drawdown Analysis'):
    """
    Genera gráfico de drawdown.
    
    Args:
        returns (Series): Serie con retornos
        title (str): Título del gráfico
        
    Returns:
        None
    """
    try:
        # Calcular retorno acumulado
        cumulative_return = (1 + returns).cumprod() - 1
        
        # Calcular drawdown
        peak = cumulative_return.cummax()
        drawdown = (cumulative_return - peak) / (1 + peak)
        
        plt.figure(figsize=(12, 6))
        
        # Graficar drawdown
        plt.fill_between(drawdown.index, drawdown.values, 0, color='red', alpha=0.3)
        plt.plot(drawdown.index, drawdown.values, color='red', alpha=0.5)
        
        # Añadir título y etiquetas
        plt.title(title)
        plt.xlabel('Date')
        plt.ylabel('Drawdown')
        plt.grid(True)
        
        # Guardar gráfico
        plt.savefig(f'./artifacts/results/figures/{title.replace(" ", "_")}.png')
        plt.close()
    
    except Exception as e:
        logging.error(f"Error generando gráfico de drawdown: {str(e)}")

def plot_regime_performance(returns, regimes, title='Performance by Regime'):
    """
    Genera gráfico de rendimiento por régimen.
    
    Args:
        returns (Series): Serie con retornos
        regimes (Series): Serie con regímenes
        title (str): Título del gráfico
        
    Returns:
        None
    """
    try:
        # Alinear fechas
        aligned_regimes = regimes.reindex(returns.index)
        
        # Crear DataFrame con retornos y regímenes
        df = pd.DataFrame({
            'returns': returns,
            'regime': aligned_regimes
        })
        
        # Calcular retorno promedio por régimen
        regime_returns = df.groupby('regime')['returns'].mean() * 252  # Anualizado
        
        plt.figure(figsize=(10, 6))
        
        # Graficar retorno por régimen
        bars = plt.bar(regime_returns.index, regime_returns.values)
        
        # Colorear barras según régimen
        colors = ['green', 'yellow', 'red']
        for i, bar in enumerate(bars):
            if i < len(colors):
                bar.set_color(colors[i])
        
        # Añadir título y etiquetas
        plt.title(title)
        plt.xlabel('Regime')
        plt.ylabel('Annualized Return')
        plt.xticks(regime_returns.index)
        plt.grid(True, axis='y')
        
        # Guardar gráfico
        plt.savefig(f'./artifacts/results/figures/{title.replace(" ", "_")}.png')
        plt.close()
    
    except Exception as e:
        logging.error(f"Error generando gráfico de rendimiento por régimen: {str(e)}")

def plot_sector_exposure(portfolio_weights, sectors, date, title='Sector Exposure'):
    """
    Genera gráfico de exposición por sector.
    
    Args:
        portfolio_weights (dict): Diccionario con pesos para una fecha
        sectors (dict): Diccionario con sectores para cada ticker
        date (datetime): Fecha para la cual mostrar exposición
        title (str): Título del gráfico
        
    Returns:
        None
    """
    try:
        # Calcular exposición por sector
        sector_exposure = {}
        
        for ticker, weight in portfolio_weights.items():
            sector = sectors.get(ticker, 'Unknown')
            sector_exposure[sector] = sector_exposure.get(sector, 0) + weight
        
        # Ordenar sectores por exposición
        sorted_sectors = sorted(sector_exposure.items(), key=lambda x: x[1], reverse=True)
        
        # Extraer sectores y exposiciones
        sector_names = [s[0] for s in sorted_sectors]
        exposures = [s[1] for s in sorted_sectors]
        
        plt.figure(figsize=(12, 6))
        
        # Graficar exposición por sector
        bars = plt.barh(sector_names, exposures)
        
        # Añadir título y etiquetas
        plt.title(f'{title} - {date.strftime("%Y-%m-%d")}')
        plt.xlabel('Exposure')
        plt.ylabel('Sector')
        plt.grid(True, axis='x')
        
        # Guardar gráfico
        plt.savefig(f'./artifacts/results/figures/{title.replace(" ", "_")}_{date.strftime("%Y%m%d")}.png')
        plt.close()
    
    except Exception as e:
        logging.error(f"Error generando gráfico de exposición por sector: {str(e)}")

def plot_volatility_targeting(market_volatility, exposure_multipliers, title='Volatility Targeting'):
    """
    Genera gráfico de volatility targeting.
    
    Args:
        market_volatility (Series): Serie con volatilidad del mercado
        exposure_multipliers (Series): Serie con multiplicadores de exposición
        title (str): Título del gráfico
        
    Returns:
        None
    """
    try:
        plt.figure(figsize=(12, 8))
        
        # Crear subplot para volatilidad
        ax1 = plt.subplot(211)
        ax1.plot(market_volatility.index, market_volatility.values, color='red')
        ax1.set_ylabel('Market Volatility (Annualized)')
        ax1.set_title('Market Volatility Over Time')
        ax1.grid(True)
        
        # Crear subplot para multiplicadores
        ax2 = plt.subplot(212, sharex=ax1)
        ax2.plot(exposure_multipliers.index, exposure_multipliers.values, color='blue')
        ax2.set_ylabel('Exposure Multiplier')
        ax2.set_title('Volatility Targeting Exposure')
        ax2.grid(True)
        
        # Ajustar diseño
        plt.tight_layout()
        
        # Guardar gráfico
        plt.savefig(f'./artifacts/results/figures/{title.replace(" ", "_")}.png')
        plt.close()
    
    except Exception as e:
        logging.error(f"Error generando gráfico de volatility targeting: {str(e)}")

        
def make_plots(strategy_returns, benchmark_returns, market_regimes, market_volatility, exposure_multipliers):        
        plot_performance(strategy_returns, benchmark_returns, title='Strategy vs Benchmark')
        plot_drawdown(strategy_returns, title='Strategy Drawdown')
        plot_regime_performance(strategy_returns, market_regimes, title='Performance by Regime')
        
        # NUEVO: Generar gráfico de volatility targeting
        plot_volatility_targeting(market_volatility, exposure_multipliers, title='Volatility Targeting')

In [9]:
import os
import logging
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from scipy import stats
from tqdm import tqdm
from datetime import datetime, timedelta
import warnings
from sklearn.linear_model import LinearRegression
import requests
from io import StringIO

# Crear directorios para resultados
os.makedirs('./artifacts/results', exist_ok=True)
os.makedirs('./artifacts/results/figures', exist_ok=True)
os.makedirs('./artifacts/results/data', exist_ok=True)

# Configurar logging
logging.basicConfig(
    filename='./artifacts/errors.txt',
    level=logging.ERROR,
    format='[%(asctime)s] %(levelname)s: %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S'
)

# Ignorar advertencias
warnings.filterwarnings('ignore')

def get_sp500_tickers():
    """
    Obtiene la lista de tickers del S&P 500 desde Wikipedia.
    
    Returns:
        dict: Diccionario con tickers como claves y sectores como valores
    """
    try:
        url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
        tables = pd.read_html(url)
        df = tables[0]
        
        # Crear diccionario de ticker -> sector
        ticker_sector_dict = dict(zip(df['Symbol'], df['GICS Sector']))
        
        # Limpiar tickers (algunos tienen puntos que yfinance no maneja bien)
        ticker_sector_dict = {ticker.replace('.', '-'): sector 
                             for ticker, sector in ticker_sector_dict.items()}
        
        return ticker_sector_dict
    
    except Exception as e:
        logging.error(f"Error obteniendo tickers del S&P 500: {str(e)}")
        # Devolver un diccionario vacío en caso de error
        return {}

def get_defensive_tickers():
    """
    Crea una lista de tickers defensivos (ETFs de bonos, oro, etc.)
    
    Returns:
        list: Lista de tickers defensivos
    """
    # ETFs de renta fija, oro, utilities y consumo defensivo
    defensive_tickers = [
        'TLT',  # iShares 20+ Year Treasury Bond ETF
        'IEF',  # iShares 7-10 Year Treasury Bond ETF
        'LQD',  # iShares iBoxx $ Investment Grade Corporate Bond ETF
        'GLD',  # SPDR Gold Shares
        'XLU',  # Utilities Select Sector SPDR Fund
        'XLP',  # Consumer Staples Select Sector SPDR Fund
        'USMV', # iShares MSCI USA Min Vol Factor ETF
        'SPLV', # Invesco S&P 500 Low Volatility ETF
        'VNQ',  # Vanguard Real Estate ETF
    ]
    
    # Crear un diccionario con sectores "defensivos" para estos tickers
    defensive_sectors = {ticker: 'Defensive' for ticker in defensive_tickers}
    
    return defensive_sectors

def download_data(tickers, start_date, end_date, include_defensive=True):
    """
    Descarga datos históricos para los tickers especificados.
    
    Args:
        tickers (list): Lista de tickers a descargar
        start_date (str): Fecha de inicio en formato 'YYYY-MM-DD'
        end_date (str): Fecha de fin en formato 'YYYY-MM-DD'
        include_defensive (bool): Si True, incluye activos defensivos
        
    Returns:
        tuple: (precios, volumen, sectores actualizados)
    """
    try:
        # Obtener tickers defensivos si se solicita
        defensive_sectors = {}
        if include_defensive:
            defensive_sectors = get_defensive_tickers()
            tickers = list(tickers) + list(defensive_sectors.keys())
        
        # Añadir un margen de tiempo para calcular características que requieren datos históricos
        extended_start = (pd.to_datetime(start_date) - pd.Timedelta(days=365)).strftime('%Y-%m-%d')
        
        # Descargar datos
        data = yf.download(tickers, start=extended_start, end=end_date, progress=False)
        
        # Extraer precios de cierre y volumen
        prices = data['Close']
        volume = data['Volume']
        
        # Verificar si hay datos
        if prices.empty or volume.empty:
            raise ValueError("No se pudieron obtener datos para los tickers especificados")
        
        # Eliminar columnas con más del 30% de valores NaN
        valid_columns = prices.columns[prices.isna().mean() < 0.3]
        prices = prices[valid_columns]
        volume = volume[valid_columns]
        
        # Llenar valores NaN con el último valor disponible
        prices = prices.fillna(method='ffill')
        volume = volume.fillna(method='ffill')
        
        # Filtrar para el período solicitado
        prices = prices.loc[start_date:end_date]
        volume = volume.loc[start_date:end_date]
        
        return prices, volume, defensive_sectors
    
    except Exception as e:
        logging.error(f"Error descargando datos: {str(e)}")
        # Devolver DataFrames vacíos en caso de error
        return pd.DataFrame(), pd.DataFrame(), {}

def calculate_returns(prices, periods):
    """
    Calcula los retornos para diferentes períodos de tiempo.
    
    Args:
        prices (DataFrame): DataFrame con precios de cierre
        periods (dict): Diccionario con nombres de períodos y número de días
        
    Returns:
        dict: Diccionario con retornos para cada período
    """
    try:
        returns = {}
        
        for period_name, days in periods.items():
            # Calcular retornos para el período especificado
            period_returns = prices.pct_change(periods=days).shift(1)
            returns[period_name] = period_returns
        
        return returns
    
    except Exception as e:
        logging.error(f"Error calculando retornos: {str(e)}")
        # Devolver un diccionario vacío en caso de error
        return {}

def calculate_volatility_target_multiplier(market_volatility, target_vol=0.20, max_leverage=1.5, min_leverage=0.25):
    """
    Calcula el multiplicador de exposición basado en la volatilidad del mercado.
    
    Args:
        market_volatility (float): Volatilidad anualizada actual del mercado
        target_vol (float): Volatilidad objetivo anualizada
        max_leverage (float): Apalancamiento máximo permitido
        min_leverage (float): Apalancamiento mínimo permitido
        
    Returns:
        float: Multiplicador para el tamaño de posición
    """
    # Evitar división por cero
    if market_volatility <= 0:
        return max_leverage
    
    # Calcular multiplicador basado en volatilidad objetivo
    vol_multiplier = target_vol / market_volatility
    
    # Limitar el multiplicador dentro del rango permitido
    vol_multiplier = max(min(vol_multiplier, max_leverage), min_leverage)
    
    return vol_multiplier

def calculate_features(prices, volume, returns):
    """
    Calcula características para cada ticker en cada fecha.
    
    Args:
        prices (DataFrame): DataFrame con precios de cierre
        volume (DataFrame): DataFrame con volumen
        returns (dict): Diccionario con retornos para diferentes períodos
        
    Returns:
        DataFrame: DataFrame con características
    """
    try:
        # Calcular retornos diarios para volatilidad
        daily_returns = prices.pct_change()
        
        # Calcular volatilidad (ventana de 21 días)
        volatility = daily_returns.rolling(window=21).std() * np.sqrt(252)
        
        # Calcular cambio de volumen (21 días)
        volume_change = volume.pct_change(periods=21)
        
        # Calcular volumen promedio (21 días)
        avg_volume = volume.rolling(window=21).mean()
        
        # Calcular autocorrelación (21 días)
        autocorr = pd.DataFrame(index=prices.index, columns=prices.columns)
        
        # Necesitamos al menos 22 días de datos para calcular autocorrelación
        min_required_days = 22
        if len(daily_returns) >= min_required_days:
            for ticker in prices.columns:
                for i in range(min_required_days, len(daily_returns)):
                    window = daily_returns.iloc[i-21:i][ticker].dropna()
                    if len(window) > 5:  # Necesitamos al menos algunos puntos para la autocorrelación
                        try:
                            autocorr.iloc[i][ticker] = window.autocorr(lag=1)
                        except:
                            autocorr.iloc[i][ticker] = 0
        
        # Calcular beta (ventana de 63 días) - NUEVO
        spy_returns = daily_returns.mean(axis=1)  # Proxy para el mercado
        beta = pd.DataFrame(index=prices.index, columns=prices.columns)
        
        for ticker in prices.columns:
            # Calcular beta usando regresión con ventana móvil
            for i in range(63, len(daily_returns)):
                x = spy_returns.iloc[i-63:i].values.reshape(-1, 1)
                y = daily_returns.iloc[i-63:i][ticker].values
                
                # Eliminar valores NaN
                mask = ~np.isnan(y)
                if sum(mask) > 30:  # Al menos 30 puntos válidos
                    try:
                        model = LinearRegression().fit(x[mask], y[mask])
                        beta.iloc[i][ticker] = model.coef_[0]
                    except:
                        beta.iloc[i][ticker] = 1.0  # Valor por defecto
        
        # Calcular indicador RSI (14 días) - NUEVO
        rsi = pd.DataFrame(index=prices.index, columns=prices.columns)
        
        for ticker in prices.columns:
            # Calcular cambios diarios
            delta = daily_returns[ticker]
            
            # Separar ganancias y pérdidas
            gains = delta.copy()
            losses = delta.copy()
            gains[gains < 0] = 0
            losses[losses > 0] = 0
            losses = abs(losses)
            
            # Calcular promedio de ganancias y pérdidas
            avg_gain = gains.rolling(window=14).mean()
            avg_loss = losses.rolling(window=14).mean()
            
            # Calcular RS y RSI
            rs = avg_gain / avg_loss
            rsi[ticker] = 100 - (100 / (1 + rs))
        
        # Asegurarse de que tenemos suficientes datos
        valid_dates = prices.index[63:]  # Cambiar de 21 a 63 debido al cálculo de beta
        
        # Crear lista para almacenar características
        features_list = []
        
        # Para cada fecha después de tener suficientes datos
        for date in valid_dates:
            # Para cada ticker
            for ticker in prices.columns:
                # Verificar si tenemos datos para este ticker en esta fecha
                if pd.isna(prices.loc[date, ticker]):
                    continue
                
                # Extraer características para este ticker en esta fecha
                features_dict = {
                    'date': date,
                    'ticker': ticker,
                    'momentum_1m': returns['1M'].loc[date, ticker] if not pd.isna(returns['1M'].loc[date, ticker]) else 0,
                    'momentum_3m': returns['3M'].loc[date, ticker] if not pd.isna(returns['3M'].loc[date, ticker]) else 0,
                    'momentum_6m': returns['6M'].loc[date, ticker] if not pd.isna(returns['6M'].loc[date, ticker]) else 0,
                    'momentum_12m': returns['12M'].loc[date, ticker] if not pd.isna(returns['12M'].loc[date, ticker]) else 0,
                    'volatility': volatility.loc[date, ticker] if not pd.isna(volatility.loc[date, ticker]) else np.nan,
                    'avg_volume': avg_volume.loc[date, ticker] if not pd.isna(avg_volume.loc[date, ticker]) else np.nan,
                    'volume_change': volume_change.loc[date, ticker] if not pd.isna(volume_change.loc[date, ticker]) else 0,
                    'autocorr': autocorr.loc[date, ticker] if not pd.isna(autocorr.loc[date, ticker]) else 0,
                    'beta': beta.loc[date, ticker] if not pd.isna(beta.loc[date, ticker]) else 1.0,
                    'rsi': rsi.loc[date, ticker] if not pd.isna(rsi.loc[date, ticker]) else 50
                }
                
                features_list.append(features_dict)
        
        # Crear DataFrame con características
        features_df = pd.DataFrame(features_list)
        
        # Manejar valores NaN
        features_df = features_df.fillna(0)
        
        return features_df
    
    except Exception as e:
        logging.error(f"Error calculando características: {str(e)}")
        # Devolver un DataFrame vacío en caso de error
        return pd.DataFrame()

def detect_market_regimes(prices, n_regimes=3):
    """
    Detecta regímenes de mercado utilizando clustering.
    
    Args:
        prices (DataFrame): DataFrame con precios de cierre
        n_regimes (int): Número de regímenes a detectar
        
    Returns:
        Series: Serie con regímenes para cada fecha
    """
    try:
        # Calcular retornos del mercado (promedio de todos los activos)
        market_returns = prices.pct_change().mean(axis=1).dropna()
        
        # Verificar si hay suficientes datos
        if len(market_returns) < 42:  # Necesitamos al menos 42 días para calcular volatilidad
            logging.warning("Datos insuficientes para detectar regímenes. Usando régimen por defecto.")
            return pd.Series(0, index=prices.index)
        
        # Calcular volatilidad rodante (21 días)
        rolling_vol = market_returns.rolling(window=21).std().dropna()
        
        # Calcular volatilidad rodante de alta frecuencia (5 días) - NUEVO
        short_vol = market_returns.rolling(window=5).std() * np.sqrt(252)
        
        # Calcular ratio de volatilidad (volatilidad reciente / volatilidad de largo plazo) - NUEVO
        vol_ratio = short_vol / rolling_vol.rolling(window=252).mean()
        vol_ratio = vol_ratio[rolling_vol.index]
        
        # Crear características para el modelo
        features = pd.DataFrame({
            'returns': market_returns[rolling_vol.index],
            'volatility': rolling_vol,
            'vol_ratio': vol_ratio[rolling_vol.index].fillna(1)  # NUEVO
        })
        
        # Normalizar características
        scaler = StandardScaler()
        features_scaled = scaler.fit_transform(features)
        
        # Aplicar K-means clustering
        kmeans = KMeans(n_clusters=n_regimes, random_state=42)
        regimes = kmeans.fit_predict(features_scaled)
        
        # Crear Serie con regímenes
        regime_series = pd.Series(regimes, index=features.index)
        
        # Calcular y ordenar los centroides para asignar regímenes de manera consistente
        # Régimen 0: baja volatilidad, Régimen 1: media volatilidad, Régimen 2: alta volatilidad
        centroids = kmeans.cluster_centers_
        vol_levels = centroids[:, 1]  # Índice 1 corresponde a la volatilidad
        sorted_indices = np.argsort(vol_levels)
        
        # Crear un mapeo de etiquetas originales a etiquetas ordenadas
        label_map = {sorted_indices[i]: i for i in range(n_regimes)}
        
        # Aplicar el mapeo a las etiquetas de régimen
        mapped_regimes = np.array([label_map[r] for r in regimes])
        regime_series = pd.Series(mapped_regimes, index=features.index)
        
        # Propagar regímenes a todas las fechas
        full_regime_series = pd.Series(index=prices.index)
        
        # Para cada fecha en el índice de precios
        for date in prices.index:
            # Si la fecha está en el índice de regímenes, usar ese régimen
            if date in regime_series.index:
                full_regime_series[date] = regime_series[date]
            # Si no, usar el último régimen disponible
            elif date > regime_series.index[0]:
                # Encontrar la fecha más reciente en el índice de regímenes
                last_date = regime_series.index[regime_series.index < date][-1]
                full_regime_series[date] = regime_series[last_date]
            # Si la fecha es anterior al primer régimen, usar el primer régimen
            else:
                full_regime_series[date] = regime_series.iloc[0]
        
        return full_regime_series
    
    except Exception as e:
        logging.error(f"Error detectando regímenes de mercado: {str(e)}")
        # Devolver una serie con régimen por defecto en caso de error
        return pd.Series(0, index=prices.index)

def generate_signals(features, market_regimes):
    """
    Genera señales de trading basadas en características y regímenes de mercado.
    
    Args:
        features (DataFrame): DataFrame con características
        market_regimes (Series): Serie con regímenes de mercado
        
    Returns:
        DataFrame: DataFrame con señales para cada ticker en cada fecha
    """
    try:
        # Crear DataFrame para almacenar señales
        signals = pd.DataFrame()
        
        # Obtener fechas únicas
        dates = features['date'].unique()
        
        # Para cada fecha
        for date in dates:
            # Obtener características para esta fecha
            date_features = features[features['date'] == date]
            
            # Obtener régimen de mercado para esta fecha
            regime = market_regimes.get(date, 0)  # Usar régimen 0 por defecto si no hay dato
            
            # Calcular señales según el régimen
            if regime == 0:  # Régimen de baja volatilidad
                # En régimen de baja volatilidad, dar más peso a momentum de largo plazo
                momentum_signal = (
                    0.1 * date_features['momentum_1m'] +
                    0.2 * date_features['momentum_3m'] +
                    0.3 * date_features['momentum_6m'] +
                    0.4 * date_features['momentum_12m']
                )
            elif regime == 1:  # Régimen de volatilidad media
                # En régimen de volatilidad media, equilibrar pesos
                momentum_signal = (
                    0.25 * date_features['momentum_1m'] +
                    0.25 * date_features['momentum_3m'] +
                    0.25 * date_features['momentum_6m'] +
                    0.25 * date_features['momentum_12m']
                )
            else:  # Régimen de alta volatilidad
                # En régimen de alta volatilidad, dar más peso a momentum de corto plazo y menor peso total a momentum
                # MODIFICADO: Incorporar señales de mean-reversion en régimen de alta volatilidad
                momentum_component = (
                    0.4 * date_features['momentum_1m'] +
                    0.3 * date_features['momentum_3m'] +
                    0.2 * date_features['momentum_6m'] +
                    0.1 * date_features['momentum_12m']
                )
                
                # Añadir componente de mean-reversion basado en RSI (sobrevendido = señal positiva)
                rsi_signal = (50 - date_features['rsi']) / 50  # Valores negativos para RSI > 50, positivos para RSI < 50
                
                # Combinar momentum y mean-reversion con más peso a mean-reversion
                momentum_signal = 0.5 * momentum_component + 0.5 * rsi_signal
            
            # Ajustar señales por volatilidad (penalizar alta volatilidad) - MODIFICADO: exponente más agresivo
            vol_adjustment = 1 / (1 + date_features['volatility'] ** 1.5)
            
            # Ajustar señales por volumen (favorecer alto volumen y cambios positivos)
            volume_adjustment = (
                date_features['avg_volume'] / date_features['avg_volume'].mean() *
                (1 + date_features['volume_change'])
            )
            volume_adjustment = volume_adjustment / volume_adjustment.mean()  # Normalizar
            
            # NUEVO: Ajustar por beta (penalizar beta alta en regímenes de alta volatilidad)
            if regime == 2:  # Alta volatilidad
                beta_adjustment = 1 / (1 + date_features['beta'])
            elif regime == 1:  # Media volatilidad
                beta_adjustment = 1 / (1 + 0.5 * date_features['beta'])
            else:  # Baja volatilidad
                beta_adjustment = 1  # Sin ajuste en baja volatilidad
            
            # Combinar señales
            combined_signal = momentum_signal * vol_adjustment * volume_adjustment * beta_adjustment
            
            # Crear DataFrame con señales para esta fecha
            date_signals = pd.DataFrame({
                'date': date,
                'ticker': date_features['ticker'],
                'signal': combined_signal,
                'autocorr': date_features['autocorr'],
                'regime': regime,
                'volatility': date_features['volatility'],
                'beta': date_features['beta'],  # NUEVO
                'rsi': date_features['rsi']     # NUEVO
            })
            
            # Añadir a DataFrame de señales
            signals = pd.concat([signals, date_signals], ignore_index=True)
        
        return signals
    
    except Exception as e:
        logging.error(f"Error generando señales: {str(e)}")
        # Devolver un DataFrame vacío en caso de error
        return pd.DataFrame()

def adjust_for_autocorrelation(signals):
    """
    Ajusta las señales basándose en la autocorrelación de los retornos.
    
    Args:
        signals (DataFrame): DataFrame con señales
        
    Returns:
        DataFrame: DataFrame con señales ajustadas
    """
    try:
        # Crear copia para no modificar el original
        adjusted_signals = signals.copy()
        
        # Para cada fecha
        for date in adjusted_signals['date'].unique():
            # Obtener señales para esta fecha
            date_signals = adjusted_signals[adjusted_signals['date'] == date]
            
            # Para cada ticker
            for idx, row in date_signals.iterrows():
                # Obtener autocorrelación
                autocorr = row['autocorr']
                
                # Ajustar señal según autocorrelación
                if not pd.isna(autocorr):
                    # Si autocorrelación es positiva, reducir la señal
                    if autocorr > 0:
                        adjustment_factor = 1 / (1 + 2 * autocorr)
                    # Si autocorrelación es negativa, aumentar la señal
                    else:
                        adjustment_factor = 1 - 2 * autocorr
                    
                    # Aplicar ajuste
                    adjusted_signals.loc[idx, 'signal'] *= adjustment_factor
        
        return adjusted_signals
    
    except Exception as e:
        logging.error(f"Error ajustando por autocorrelación: {str(e)}")
        # Devolver señales sin ajustar en caso de error
        return signals

def combine_signals(signals, lookback_window=63):
    """
    Combina señales utilizando pesos adaptativos basados en rendimiento reciente.
    Versión optimizada para mejorar rendimiento.
    
    Args:
        signals (DataFrame): DataFrame con señales
        lookback_window (int): Ventana para evaluar rendimiento de señales
        
    Returns:
        DataFrame: DataFrame con señales combinadas
    """
    try:
        # Si el DataFrame está vacío, devolverlo sin cambios
        if signals.empty:
            return signals
        
        # Crear copia para no modificar el original
        combined_signals = signals.copy()
        
        # Obtener fechas únicas ordenadas
        dates = np.sort(combined_signals['date'].unique())
        
        # Si no hay suficientes fechas, devolver señales sin combinar
        if len(dates) <= lookback_window:
            return combined_signals
        
        # Precomputar un diccionario de mapeo de régimen a factor de exposición
        regime_exposure_map = {0: 1.0, 1: 0.8, 2: 0.4}
        
        # Crear un índice para acceso rápido
        combined_signals.set_index(['date', 'ticker'], inplace=True, drop=False)
        
        # Preprocesar todas las señales para evitar filtrados repetidos
        date_dict = {}
        for date in dates:
            try:
                date_dict[date] = combined_signals.xs(date, level='date')
            except:
                date_dict[date] = pd.DataFrame()  # Fecha sin datos
        
        # Para cada fecha después de la ventana de lookback
        for i in range(lookback_window, len(dates)):
            current_date = dates[i]
            
            # Obtener señales para la fecha actual (ya preprocesado)
            current_signals = date_dict[current_date]
            
            # Si no hay señales para esta fecha, continuar
            if current_signals.empty:
                continue
            
            # Obtener régimen actual (una sola vez)
            current_regime = current_signals['regime'].iloc[0]
            
            # Obtener factor global según régimen (una sola vez)
            regime_exposure_factor = regime_exposure_map.get(current_regime, 1.0)
            
            # Crear un array para todas las fechas de lookback
            lookback_dates = dates[i-lookback_window:i]
            
            # Para cada ticker, calcular de forma vectorizada
            processed_tickers = set()
            
            # Procesar en batch por ticker
            for ticker in current_signals['ticker'].unique():
                processed_tickers.add(ticker)
                
                # Recopilar historial para este ticker (de forma eficiente)
                ticker_history_list = []
                for ld in lookback_dates:
                    if ld in date_dict and not date_dict[ld].empty:
                        try:
                            ticker_data = date_dict[ld].xs(ticker, level='ticker', drop=False)
                            if not ticker_data.empty:
                                ticker_history_list.append(ticker_data)
                        except:
                            continue
                
                if not ticker_history_list:
                    continue
                    
                ticker_history = pd.concat(ticker_history_list)
                
                # Verificar suficientes datos
                if len(ticker_history) < lookback_window / 2:
                    continue
                
                try:
                    # Calcular correlación de forma más eficiente
                    signal_regime_corr = np.corrcoef(
                        ticker_history['signal'].values,
                        ticker_history['regime'].values
                    )[0, 1]
                    
                    # Verificar validez
                    if np.isnan(signal_regime_corr):
                        continue
                    
                    # Obtener régimen actual para este ticker
                    ticker_regime = current_signals.loc[
                        current_signals['ticker'] == ticker, 'regime'
                    ].iloc[0]
                    
                    # Calcular factor según correlación
                    if signal_regime_corr > 0:
                        regime_factor = 1 + 0.2 * ticker_regime
                    else:
                        regime_factor = 1 - 0.2 * ticker_regime
                    
                    # Calcular factor combinado (una sola multiplicación)
                    combined_factor = regime_factor * regime_exposure_factor
                    
                    # Aplicar factor de forma más eficiente (una sola operación)
                    idx = (current_date, ticker)
                    if idx in combined_signals.index:
                        combined_signals.at[idx, 'signal'] *= combined_factor
                except Exception as inner_e:
                    # Manejar errores silenciosamente para continuar el procesamiento
                    continue
        
        # Restaurar el DataFrame a su forma original
        combined_signals.reset_index(drop=True, inplace=True)
        
        return combined_signals
    
    except Exception as e:
        logging.error(f"Error combinando señales: {str(e)}")
        # Devolver señales sin combinar en caso de error
        return signals

def construct_portfolio(signals, sectors, date, top_pct=0.1, max_sector_exposure=0.25, max_position_size=0.05):
    """
    Construye un portafolio basado en señales para una fecha específica.
    
    Args:
        signals (DataFrame): DataFrame con señales
        sectors (dict): Diccionario con sectores para cada ticker
        date (datetime): Fecha para la cual construir el portafolio
        top_pct (float): Porcentaje de tickers con mejores señales a incluir
        max_sector_exposure (float): Exposición máxima por sector
        max_position_size (float): Tamaño máximo por posición
        
    Returns:
        dict: Diccionario con pesos para cada ticker
    """
    try:
        # Obtener señales para la fecha especificada
        date_signals = signals[signals['date'] == date].copy()
        
        # Si no hay señales para esta fecha, devolver diccionario vacío
        if date_signals.empty:
            return {}
        
        # Añadir sector a cada ticker
        date_signals['sector'] = date_signals['ticker'].map(lambda x: sectors.get(x, 'Unknown'))
        
        # Obtener el régimen actual
        current_regime = date_signals['regime'].iloc[0]
        
        # NUEVO: Separar tickers defensivos y no defensivos
        defensive_tickers = date_signals[date_signals['sector'] == 'Defensive']
        non_defensive_tickers = date_signals[date_signals['sector'] != 'Defensive']
        
        # NUEVO: Ajustar porcentaje de exposición a activos defensivos según régimen
        defensive_allocation = 0.0  # Por defecto, sin activos defensivos
        
        if current_regime == 2:  # Alta volatilidad
            defensive_allocation = 0.40  # 40% a defensivos en alta volatilidad
        elif current_regime == 1:  # Media volatilidad
            defensive_allocation = 0.20  # 20% a defensivos en media volatilidad
        elif current_regime == 0:  # Baja volatilidad
            defensive_allocation = 0.05  # 5% a defensivos en baja volatilidad
        
        # Asignar pesos iniciales
        weights = {}
        
        # Procesar activos no defensivos
        non_defensive_tickers = non_defensive_tickers.sort_values('signal', ascending=False)
        n_non_defensive = int(len(non_defensive_tickers) * top_pct)
        top_non_defensive = non_defensive_tickers.head(n_non_defensive)
        
        # Calcular exposición por sector para activos no defensivos
        sector_exposure = top_non_defensive.groupby('sector').size() / n_non_defensive
        
        # NUEVO: Incorporar risk parity para activos no defensivos
        # Inversamente proporcional a la volatilidad
        top_non_defensive['risk_weight'] = 1 / top_non_defensive['volatility'].replace(0, 0.01)
        top_non_defensive['risk_weight'] = top_non_defensive['risk_weight'] / top_non_defensive['risk_weight'].sum()
        
        # Combinar señal y riesgo (70% señal, 30% risk parity)
        top_non_defensive['combined_weight'] = 0.7 * top_non_defensive['signal'] + 0.3 * top_non_defensive['risk_weight']
        
        # Para cada ticker no defensivo
        for _, row in top_non_defensive.iterrows():
            ticker = row['ticker']
            sector = row['sector']
            
            # Si la exposición del sector excede el máximo, reducir peso
            if sector in sector_exposure and sector_exposure[sector] > max_sector_exposure:
                weight = row['combined_weight'] * (max_sector_exposure / sector_exposure[sector])
            else:
                weight = row['combined_weight']
            
            weights[ticker] = weight * (1 - defensive_allocation)
        
        # Procesar activos defensivos
        if not defensive_tickers.empty and defensive_allocation > 0:
            defensive_tickers = defensive_tickers.sort_values('signal', ascending=False)
            
            # Tomar los mejores activos defensivos
            n_defensive = min(5, len(defensive_tickers))
            top_defensive = defensive_tickers.head(n_defensive)
            
            # Asignar pesos equitativos a los activos defensivos
            defensive_weight_per_asset = defensive_allocation / n_defensive
            
            for ticker in top_defensive['ticker']:
                weights[ticker] = defensive_weight_per_asset
        
        # NUEVO: Limitar tamaño máximo por posición
        for ticker in list(weights.keys()):
            if weights[ticker] > max_position_size:
                excess = weights[ticker] - max_position_size
                weights[ticker] = max_position_size
                
                # Redistribuir exceso proporcionalmente entre otros activos
                total_other_weights = sum(w for t, w in weights.items() if t != ticker)
                if total_other_weights > 0:
                    for t in weights:
                        if t != ticker:
                            weights[t] += excess * (weights[t] / total_other_weights)
        
        # Normalizar pesos para que sumen 1
        total_weight = sum(weights.values())
        
        if total_weight > 0:
            weights = {ticker: weight / total_weight for ticker, weight in weights.items()}
        
        return weights
    
    except Exception as e:
        logging.error(f"Error construyendo portafolio para {date}: {str(e)}")
        # Devolver diccionario vacío en caso de error
        return {}

def implement_stop_loss(prices, portfolio_weights, current_date, max_drawdown=-0.10):
    """
    Implementa stop loss para reducir drawdowns.
    
    Args:
        prices (DataFrame): DataFrame con precios de cierre
        portfolio_weights (dict): Diccionario con pesos para cada fecha
        current_date (datetime): Fecha actual
        max_drawdown (float): Drawdown máximo permitido
        
    Returns:
        dict: Diccionario con pesos ajustados
    """
    try:
        # Si no hay pesos para la fecha actual, devolver diccionario vacío
        if current_date not in portfolio_weights:
            return {}
        
        # Obtener pesos actuales
        current_weights = portfolio_weights[current_date].copy()
        
        # Si no hay pesos, devolver diccionario vacío
        if not current_weights:
            return {}
        
        # Obtener fechas anteriores (último mes)
        one_month_ago = current_date - pd.Timedelta(days=30)
        past_dates = prices.index[(prices.index >= one_month_ago) & (prices.index < current_date)]
        
        if len(past_dates) < 5:  # Necesitamos al menos algunos días de historia
            return current_weights
        
        # Calcular drawdowns para cada activo
        for ticker in list(current_weights.keys()):
            if ticker in prices.columns:
                # Obtener precios para este ticker
                ticker_prices = prices.loc[past_dates, ticker]
                
                if len(ticker_prices) < 5:
                    continue
                
                # Calcular drawdown
                peak = ticker_prices.max()
                current_price = prices.loc[current_date, ticker]
                drawdown = (current_price / peak) - 1
                
                # Si drawdown excede el máximo, eliminar posición
                if drawdown < max_drawdown:
                    del current_weights[ticker]
        
        # Si se eliminaron todas las posiciones, devolver diccionario vacío
        if not current_weights:
            return {}
        
        # Normalizar pesos restantes
        total_weight = sum(current_weights.values())
        if total_weight > 0:
            current_weights = {ticker: weight / total_weight for ticker, weight in current_weights.items()}
        
        return current_weights
    
    except Exception as e:
        logging.error(f"Error implementando stop loss para {current_date}: {str(e)}")
        # Devolver los pesos originales en caso de error
        return portfolio_weights.get(current_date, {})

def calculate_portfolio_returns(prices, portfolio_weights, start_date, end_date):
    """
    Calcula los retornos del portafolio.
    
    Args:
        prices (DataFrame): DataFrame con precios de cierre
        portfolio_weights (dict): Diccionario con pesos para cada fecha
        start_date (datetime): Fecha de inicio
        end_date (datetime): Fecha de fin
        
    Returns:
        Series: Serie con retornos del portafolio
    """
    try:
        # Crear serie para almacenar retornos
        strategy_returns = pd.Series(index=pd.date_range(start=start_date, end=end_date, freq='B'))
        
        # Filtrar fechas de trading disponibles
        trading_dates = prices.index
        trading_dates = trading_dates[(trading_dates >= start_date) & (trading_dates <= end_date)]
        
        if len(trading_dates) < 2:
            logging.warning("Insuficientes fechas de trading para calcular retornos")
            return pd.Series(index=pd.date_range(start=start_date, end=end_date, freq='B'))
        
        # Verificar si hay pesos para alguna fecha
        valid_dates = [d for d in trading_dates if d in portfolio_weights]
        if not valid_dates:
            logging.warning("No hay pesos de portafolio para ninguna fecha en el período")
            return pd.Series(index=pd.date_range(start=start_date, end=end_date, freq='B'))
        
        # Calcular retornos diarios
        daily_returns = prices.pct_change()
        
        # Inicializar pesos actuales
        current_weights = None
        last_rebalance_date = None
        
        # Para cada fecha de trading
        for i in range(1, len(trading_dates)):
            current_date = trading_dates[i]
            previous_date = trading_dates[i-1]
            
            # Si es fecha de rebalanceo o primera fecha, actualizar pesos
            if current_date in portfolio_weights:
                current_weights = portfolio_weights[current_date]
                last_rebalance_date = current_date
            
            # Si no hay pesos actuales, continuar
            if current_weights is None:
                continue
            
            # Calcular retorno del portafolio para esta fecha
            portfolio_return = 0
            
            for ticker, weight in current_weights.items():
                # Verificar si el ticker está en los datos
                if ticker in daily_returns.columns:
                    # Obtener retorno para este ticker
                    ticker_return = daily_returns.loc[current_date, ticker]
                    
                    # Si no es NaN, añadir al retorno del portafolio
                    if not pd.isna(ticker_return):
                        portfolio_return += weight * ticker_return
            
            # Guardar retorno del portafolio
            strategy_returns[current_date] = portfolio_return
        
        # Eliminar NaN
        strategy_returns = strategy_returns.dropna()
        
        return strategy_returns
    
    except Exception as e:
        logging.error(f"Error calculando retornos del portafolio: {str(e)}")
        # Devolver serie vacía en caso de error
        return pd.Series()

def calculate_performance_metrics(returns, benchmark_returns=None):
    """
    Calcula métricas de rendimiento para una serie de retornos.
    
    Args:
        returns (Series): Serie con retornos
        benchmark_returns (Series, optional): Serie con retornos del benchmark
        
    Returns:
        dict: Diccionario con métricas de rendimiento
    """
    try:
        # Verificar si hay retornos
        if returns is None or len(returns) == 0:
            return {
                'annualized_return': 0,
                'annualized_volatility': 0,
                'sharpe_ratio': 0,
                'max_drawdown': 0,
                'win_rate': 0,
                'information_ratio': 0,
                'sortino_ratio': 0,  # NUEVO
                'calmar_ratio': 0,   # NUEVO
                'avg_monthly_return': 0,  # NUEVO
                'worst_month': 0     # NUEVO
            }
        
        # Asegurarnos de que trabajamos con una Serie unidimensional
        if isinstance(returns, pd.DataFrame):
            if returns.shape[1] == 1:
                returns = returns.iloc[:, 0]
            else:
                returns = returns.mean(axis=1)  # Promedio si hay múltiples columnas
        
        # Eliminar NaNs
        returns_clean = returns.dropna()
        
        # Si después de eliminar NaNs no quedan datos, devolver ceros
        if len(returns_clean) == 0:
            return {
                'annualized_return': 0,
                'annualized_volatility': 0,
                'sharpe_ratio': 0,
                'max_drawdown': 0,
                'win_rate': 0,
                'information_ratio': 0,
                'sortino_ratio': 0,
                'calmar_ratio': 0,
                'avg_monthly_return': 0,
                'worst_month': 0
            }
        
        # Calcular retorno acumulado
        cumulative_return = (1 + returns_clean).cumprod() - 1
        
        # Calcular retorno anualizado
        n_years = len(returns_clean) / 252.0
        annualized_return = (1 + cumulative_return.iloc[-1]) ** (1 / n_years) - 1
        
        # Calcular volatilidad anualizada
        annualized_volatility = returns_clean.std() * np.sqrt(252)
        
        # Calcular Sharpe ratio
        risk_free_rate = 0.02  # Tasa libre de riesgo (2%)
        sharpe_ratio = (annualized_return - risk_free_rate) / annualized_volatility if annualized_volatility > 0 else 0
        
        # Calcular máximo drawdown
        peak = cumulative_return.cummax()
        drawdown = (cumulative_return - peak) / (1 + peak)
        max_drawdown = drawdown.min()
        
        # Calcular win rate
        win_rate = (returns_clean > 0).mean()
        
        # NUEVO: Calcular Sortino ratio (solo considera downside volatility)
        downside_returns = returns_clean[returns_clean < 0]
        downside_volatility = downside_returns.std() * np.sqrt(252) if len(downside_returns) > 0 else 0
        sortino_ratio = (annualized_return - risk_free_rate) / downside_volatility if downside_volatility > 0 else 0
        
        # NUEVO: Calcular Calmar ratio (retorno anualizado / máximo drawdown absoluto)
        calmar_ratio = annualized_return / abs(max_drawdown) if max_drawdown < 0 else 0
        
        # NUEVO: Calcular retorno mensual promedio y peor mes
        monthly_returns = returns_clean.resample('M').apply(lambda x: (1 + x).prod() - 1)
        avg_monthly_return = monthly_returns.mean()
        worst_month = monthly_returns.min()
        
        # Calcular Information Ratio si hay benchmark
        information_ratio = 0
        if benchmark_returns is not None and hasattr(benchmark_returns, '__len__') and len(benchmark_returns) > 0:
            # Convertir a Serie unidimensional si es necesario
            if isinstance(benchmark_returns, pd.DataFrame):
                if benchmark_returns.shape[1] == 1:
                    benchmark_returns = benchmark_returns.iloc[:, 0]
                else:
                    benchmark_returns = benchmark_returns.mean(axis=1)
            elif isinstance(benchmark_returns, np.ndarray):
                if benchmark_returns.ndim > 1:
                    # Si es un array de más de 1 dimensión, aplanarlo
                    benchmark_returns = benchmark_returns.flatten()
                benchmark_returns = pd.Series(benchmark_returns, index=returns.index[:len(benchmark_returns)])
            
            # Limpiar benchmark
            benchmark_clean = pd.Series(benchmark_returns).dropna()
            
            if len(benchmark_clean) > 0:
                # Intentar alinear índices
                try:
                    common_dates = returns_clean.index.intersection(benchmark_clean.index)
                    if len(common_dates) > 0:
                        aligned_returns = returns_clean.loc[common_dates]
                        aligned_benchmark = benchmark_clean.loc[common_dates]
                        
                        # Calcular excess returns
                        excess_returns = aligned_returns - aligned_benchmark
                        
                        # Calcular tracking error
                        tracking_error = excess_returns.std() * np.sqrt(252)
                        
                        # Calcular Information Ratio
                        if tracking_error > 0:
                            information_ratio = excess_returns.mean() * 252 / tracking_error
                except Exception as inner_e:
                    logging.error(f"Error alineando benchmark: {str(inner_e)}")
        
        return {
            'annualized_return': float(annualized_return),
            'annualized_volatility': float(annualized_volatility),
            'sharpe_ratio': float(sharpe_ratio),
            'max_drawdown': float(max_drawdown),
            'win_rate': float(win_rate),
            'information_ratio': float(information_ratio),
            'sortino_ratio': float(sortino_ratio),
            'calmar_ratio': float(calmar_ratio),
            'avg_monthly_return': float(avg_monthly_return),
            'worst_month': float(worst_month)
        }
    
    except Exception as e:
        logging.error(f"Error calculando métricas de rendimiento: {str(e)}")
        import traceback
        logging.error(traceback.format_exc())
        # Devolver métricas vacías en caso de error
        return {
            'annualized_return': 0,
            'annualized_volatility': 0,
            'sharpe_ratio': 0,
            'max_drawdown': 0,
            'win_rate': 0,
            'information_ratio': 0,
            'sortino_ratio': 0,
            'calmar_ratio': 0,
            'avg_monthly_return': 0,
            'worst_month': 0
        }



def backtest_strategy(tickers, sectors, start_date, end_date, rebalance_freq='M'):
    """
    Realiza un backtest de la estrategia.
    
    Args:
        tickers (list): Lista de tickers
        sectors (dict): Diccionario con sectores para cada ticker
        start_date (str): Fecha de inicio en formato 'YYYY-MM-DD'
        end_date (str): Fecha de fin en formato 'YYYY-MM-DD'
        rebalance_freq (str): Frecuencia de rebalanceo ('D', 'W', 'M', etc.)
        
    Returns:
        tuple: (retornos de la estrategia, retornos del benchmark, métricas, regímenes de mercado, 
                volatilidad del mercado, multiplicadores de exposición, pesos del último portafolio)
    """
    try:
        print("Iniciando backtest...")
        
        # Descargar datos incluyendo activos defensivos
        prices, volume, defensive_sectors = download_data(tickers, start_date, end_date, include_defensive=True)
        
        # Actualizar diccionario de sectores con activos defensivos
        sectors.update(defensive_sectors)
        
        # Verificar si hay datos
        if prices.empty or volume.empty:
            raise ValueError("No se pudieron obtener datos para los tickers especificados")
        
        # Descargar datos del benchmark (S&P 500)
        benchmark_data = yf.download('^GSPC', start=start_date, end=end_date, progress=False)
        benchmark_returns = benchmark_data['Close'].pct_change().dropna()
        
        # Calcular retornos para diferentes períodos
        periods = {
            '1M': 21,
            '3M': 63,
            '6M': 126,
            '12M': 252
        }
        returns = calculate_returns(prices, periods)
        
        # Calcular características
        print("Calculando características...")
        features = calculate_features(prices, volume, returns)
        
        # Detectar regímenes de mercado
        print("Detectando regímenes de mercado...")
        market_regimes = detect_market_regimes(prices)
        
        # Generar señales
        print("Generando señales...")
        signals = combine_signals(features, market_regimes)
        
        # Ajustar señales por autocorrelación
        print("Ajustando señales por autocorrelación...")
        adjusted_signals = adjust_for_autocorrelation(signals)
        
        # Combinar señales
        print("Combinando señales...")
        combined_signals = combine_signals(adjusted_signals)
        
        # Determinar fechas de rebalanceo
        rebalance_dates = pd.date_range(start=start_date, end=end_date, freq=rebalance_freq)
        rebalance_dates = rebalance_dates[rebalance_dates.isin(prices.index)]
        
        # NUEVO: Calcular volatilidad del mercado para volatility targeting
        market_returns = prices.pct_change().mean(axis=1)
        market_volatility = market_returns.rolling(window=21).std() * np.sqrt(252)
        
        # NUEVO: Calcular multiplicadores de exposición para volatility targeting
        exposure_multipliers = pd.Series(index=prices.index)
        for date in prices.index:
            if date in market_volatility.index and not pd.isna(market_volatility[date]):
                exposure_multipliers[date] = calculate_volatility_target_multiplier(market_volatility[date])
            else:
                exposure_multipliers[date] = 1.0
        
        # Construir portafolios para cada fecha de rebalanceo
        print("Construyendo portafolios...")
        portfolio_weights = {}
        
        for date in rebalance_dates:
            # Verificar si hay señales para esta fecha
            date_signals = combined_signals[combined_signals['date'] == date]
            
            if not date_signals.empty:
                # Construir portafolio
                weights = construct_portfolio(combined_signals, sectors, date, max_position_size=0.05)
                
                # NUEVO: Implementar stop loss
                # weights = implement_stop_loss(prices, {date: weights}, date, max_drawdown=-0.15)
                
                # NUEVO: Aplicar volatility targeting
                if date in exposure_multipliers.index:
                    vol_multiplier = exposure_multipliers[date]
                    
                    # Si el multiplicador es menor que 1, reducir exposición total
                    if vol_multiplier < 1:
                        # Crear posición en efectivo (no invertida)
                        weights['CASH'] = 1 - vol_multiplier
                        
                        # Reducir todas las demás posiciones proporcionalmente
                        for ticker in list(weights.keys()):
                            if ticker != 'CASH':
                                weights[ticker] *= vol_multiplier
                
                # Guardar pesos
                portfolio_weights[date] = weights
        
        # Calcular retornos del portafolio
        print("Calculando retornos...")
        strategy_returns = calculate_portfolio_returns(
            prices,
            portfolio_weights,
            pd.to_datetime(start_date),
            pd.to_datetime(end_date)
        )
        
        # Calcular métricas de rendimiento
        print("Calculando métricas de rendimiento...")
        metrics = calculate_performance_metrics(strategy_returns, benchmark_returns)
        
        
        # Guardar métricas en CSV
        metrics_df = pd.DataFrame([metrics])
        metrics_df.to_csv('./artifacts/results/data/performance_metrics.csv', index=False)
        
        # Guardar retornos en CSV
        strategy_returns.to_csv('./artifacts/results/data/strategy_returns.csv')
        benchmark_returns.to_csv('./artifacts/results/data/benchmark_returns.csv')
        
        # Variable para almacenar los pesos del último portafolio
        last_portfolio_weights = {}
        
        # Guardar exposición sectorial para la última fecha de rebalanceo
        if rebalance_dates.size > 0:
            last_rebalance = rebalance_dates[-1]
            if last_rebalance in portfolio_weights:
                plot_sector_exposure(
                    portfolio_weights[last_rebalance],
                    sectors,
                    last_rebalance,
                    title='Last Rebalance Sector Exposure'
                )
                
                # Guardar los pesos del último portafolio
                last_portfolio_weights = portfolio_weights[last_rebalance]
                
                # Convertir el diccionario de pesos a DataFrame y guardar como CSV
                last_weights_df = pd.DataFrame(list(last_portfolio_weights.items()), 
                                             columns=['ticker', 'weight'])
                last_weights_df['date'] = last_rebalance
                last_weights_df = last_weights_df[['date', 'ticker', 'weight']]  # Reordenar columnas
                last_weights_df.to_csv(f'./artifacts/results/data/last_portfolio_weights{end_date}.csv', index=False)
                
                print(f"Pesos del último portafolio guardados (fecha: {last_rebalance.strftime('%Y-%m-%d')})")
        
        print("Backtest completado.")
        
        return strategy_returns, benchmark_returns, metrics, market_regimes, market_volatility, exposure_multipliers, last_portfolio_weights
    
    except Exception as e:
        logging.error(f"Error en backtest: {str(e)}")
        import traceback
        logging.error(traceback.format_exc())
        # Devolver valores vacíos en caso de error
        return pd.Series(), pd.Series(), {}, pd.Series(), pd.Series(), pd.Series(), {}


def main():
    """
    Función principal que ejecuta la estrategia.
    """
    try:
        print("Iniciando estrategia de momentum multi-horizonte mejorada...")
        
        # Obtener tickers y sectores del S&P 500
        sectors = get_sp500_tickers()
        tickers = list(sectors.keys())
        
        # Si hay demasiados tickers, limitar para evitar errores de API
        if len(tickers) > 10:
            tickers = tickers[:10]
            sectors = {ticker: sectors[ticker] for ticker in tickers}
        
        # Definir fechas
        end_date = datetime.now().strftime('%Y-%m-%d')
        start_date = (datetime.now() - timedelta(days=3*365)).strftime('%Y-%m-%d')  # 3 años
        # start_date = "2022-01-01"
        # end_date = "2024-01-01"

        # Realizar backtest
        print("\n=== Ejecutando Backtest ===", start_date, end_date)
    
        strategy_returns, benchmark_returns, metrics, market_regimes, market_volatility, exposure_multipliers, last_portfolio_weights = backtest_strategy(
            tickers,
            sectors,
            start_date,
            end_date,
            rebalance_freq='M'  # Rebalanceo mensual
        )
        
        # Mostrar métricas
        print("\nMétricas de rendimiento del backtest:")
        for metric, value in metrics.items():
            print(f"{metric}: {value:.4f}")

        # Make plots
        print("\nPlots:")
        make_plots(strategy_returns, benchmark_returns, market_regimes, market_volatility, exposure_multipliers)
        
        # Mostrar los pesos del último portafoliou
        if last_portfolio_weights:
            print("\nPesos del último portafolio:")
            for ticker, weight in sorted(last_portfolio_weights.items(), key=lambda x: x[1], reverse=True):
                print(f"{ticker}: {weight:.4f}")
            print(f"\nEstos pesos han sido guardados en './artifacts/results/data/last_portfolio_weights{end_date}.csv'")
                   
        print("\nEstrategia completada. Resultados guardados en ./artifacts/results/")
    
    except Exception as e:
        logging.error(f"Error en función principal: {str(e)}")
        import traceback
        logging.error(traceback.format_exc())
        print(f"Error: {str(e)} {traceback.format_exc()}")

if __name__ == "__main__":
    main()


Iniciando estrategia de momentum multi-horizonte mejorada...

=== Ejecutando Backtest === 2022-04-21 2025-04-20
Iniciando backtest...
Calculando características...
Detectando regímenes de mercado...
Generando señales...
Ajustando señales por autocorrelación...
Combinando señales...
Construyendo portafolios...
Calculando retornos...
Calculando métricas de rendimiento...
Pesos del último portafolio guardados (fecha: 2025-03-31)
Backtest completado.

Métricas de rendimiento del backtest:
annualized_return: 0.0000
annualized_volatility: 0.0000
sharpe_ratio: 0.0000
max_drawdown: 0.0000
win_rate: 0.0000
information_ratio: -0.7109
sortino_ratio: 0.0000
calmar_ratio: 0.0000
avg_monthly_return: 0.0000
worst_month: 0.0000

Plots:

Estrategia completada. Resultados guardados en ./artifacts/results/


In [6]:
print("DONE")

DONE
